In [1]:
!pip install simpletransformers
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from typing import Dict, List, Any
import logging
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
import jsonlines
from simpletransformers.classification import ClassificationModel, ClassificationArgs

import random 
import numpy as np
import torch

def random_seed(seed_value): 
    np.random.seed(seed_value) 
    torch.manual_seed(seed_value)
    random.seed(seed_value) 


random_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.8/249.8 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 16.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 33.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=db0608e3e6af5c689f7cb2b5b03a725ec5fab63642d496d0795946b7db63300b
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=d2e72e4df8a77087fc0e60e218027451561e7fb7dc0fc47fe76fa17f22f557fc
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9c

In [2]:
def read_give_instances(path):
    
    texts = []
    labels = []
    with jsonlines.open(path) as f_in:
        for json_object in f_in:
            text=json_object.get('text')
            label=json_object.get('label')
            metadata=json_object.get('metadata')
            texts.append(text)
            labels.append(label)
            
    return texts, labels

In [3]:
train_texts, train_labels = read_give_instances("../input/scibert-dataset/text_classification/chemprot/train.txt")
dev_texts, dev_labels = read_give_instances("../input/scibert-dataset/text_classification/chemprot/dev.txt")
test_texts, test_labels = read_give_instances("../input/scibert-dataset/text_classification/chemprot/test.txt")

In [4]:
np.unique(train_labels)

array(['ACTIVATOR', 'AGONIST', 'AGONIST-ACTIVATOR', 'AGONIST-INHIBITOR',
       'ANTAGONIST', 'DOWNREGULATOR', 'INDIRECT-DOWNREGULATOR',
       'INDIRECT-UPREGULATOR', 'INHIBITOR', 'PRODUCT-OF', 'SUBSTRATE',
       'SUBSTRATE_PRODUCT-OF', 'UPREGULATOR'], dtype='<U22')

In [5]:
train_texts[4], train_labels[:5]

('<< Epidermal growth factor receptor >> inhibitors currently under investigation include the small molecules gefitinib (Iressa, ZD1839) and erlotinib ([[ Tarceva ]], OSI-774), as well as monoclonal antibodies such as cetuximab (IMC-225, Erbitux).',
 ['INHIBITOR', 'INHIBITOR', 'INHIBITOR', 'INHIBITOR', 'INHIBITOR'])

In [6]:
df_train = pd.DataFrame({"text":train_texts,"labels":train_labels})
df_val = pd.DataFrame({"text":dev_texts,"labels":dev_labels})
df_test = pd.DataFrame({"text":test_texts,"labels":test_labels})
df_train.head()

,text,labels
0,<< Epidermal growth factor receptor >> inhibit...,INHIBITOR
1,<< Epidermal growth factor receptor >> inhibit...,INHIBITOR
2,<< Epidermal growth factor receptor >> inhibit...,INHIBITOR
3,<< Epidermal growth factor receptor >> inhibit...,INHIBITOR
4,<< Epidermal growth factor receptor >> inhibit...,INHIBITOR


In [7]:
len(df_train['labels'].unique())

13

In [8]:
lb = LabelEncoder()

df_train['labels'] = lb.fit_transform(df_train['labels'])
df_val['labels'] = lb.transform(df_val['labels'])
df_test['labels'] = lb.transform(df_test['labels'])
df_train.head()

,text,labels
0,<< Epidermal growth factor receptor >> inhibit...,8
1,<< Epidermal growth factor receptor >> inhibit...,8
2,<< Epidermal growth factor receptor >> inhibit...,8
3,<< Epidermal growth factor receptor >> inhibit...,8
4,<< Epidermal growth factor receptor >> inhibit...,8


In [9]:
len(df_train['labels'].unique())

13

In [10]:
model_args = ClassificationArgs(eval_batch_size=64,
                               learning_rate = 2e-5,
                               max_seq_length = 256,
                               no_save = True,
                               num_train_epochs = 4,
                               train_batch_size = 32)

In [11]:
model = ClassificationModel("bert","AnonymousSub/fpdm_models_scibert_hybrid_epochs_4",args = model_args,num_labels=13)

Downloading:   0%|          | 0.00/706 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at AnonymousSub/fpdm_models_scibert_hybrid_epochs_4 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/427 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [12]:
model.train_model(df_train,eval_df = df_val)

  0%|          | 0/4169 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/131 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/131 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/131 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/131 [00:00<?, ?it/s]

(524, 0.7035870176405852)

In [13]:
predictions, raw_outputs = model.predict(df_test['text'].tolist())

  0%|          | 0/3469 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

In [14]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score

In [15]:
print(classification_report(df_test['labels'], predictions))

              precision    recall  f1-score   support

           0       0.74      0.86      0.80       292
           1       0.76      0.88      0.82       182
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00        12
           4       0.87      0.89      0.88       293
           5       0.11      0.03      0.04        72
           6       0.72      0.84      0.77       340
           7       0.82      0.73      0.78       334
           8       0.91      0.93      0.92      1255
           9       0.69      0.68      0.69       191
          10       0.85      0.82      0.83       453
          12       0.00      0.00      0.00        41

    accuracy                           0.83      3469
   macro avg       0.54      0.55      0.54      3469
weighted avg       0.81      0.83      0.81      3469



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
print(accuracy_score(df_test['labels'], predictions))
print(precision_score(df_test['labels'], predictions,average='micro'))
print(recall_score(df_test['labels'], predictions,average='micro'))
print(f1_score(df_test['labels'], predictions,average='micro'))

0.8273277601614298
0.8273277601614298
0.8273277601614298
0.8273277601614298
